Crypto Trading using FINRL Framework

In [68]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn import preprocessing

matplotlib.use("Agg")

import ccxt
import pandas
import os
from pathlib import Path
import sys
import csv
import datetime

from finrl.apps import config
from finrl.neo_finrl.preprocessor.yahoodownloader import YahooDownloader
from finrl.neo_finrl.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.neo_finrl.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.neo_finrl.data_processors import processor_ccxt
from finrl.neo_finrl.preprocessor.binancedownloader import *
import finrl.neo_finrl.preprocessor.binancedownloader
from finrl.neo_finrl.data_processors import processor_ccxt
from finrl.neo_finrl.env_cryptocurrency_trading.env_btc_ccxt import BitcoinEnv


import itertools

In [39]:
#Download Data and save to /data

scrape_candles_to_csv('dot_usdt_15m.csv', 'binance', 3, 'DOT/USDT', '15m', '2021-08-0100:00:00Z',limit=1000)

982 DOT/USDT candles in total from 2021-08-10T03:15:00.000Z to 2021-08-20T13:00:00.000Z
1982 DOT/USDT candles in total from 2021-07-30T17:15:00.000Z to 2021-08-20T13:00:00.000Z


NameError: name 'Path' is not defined

In [13]:
data_processor = processor_ccxt.CCXTEngineer()
data = data_processor.data_fetch('20210101 01:00:00','20210820 01:00:00',period='15m',pair_list=['DOT/USDT'])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
31995
32990
33985
34980
35975
36970
37965
38960
39955
40950
41945
42945
43945
44945
45945
46945
47945
48945
49945
50945
51945
52945
53945
54939
55933
56927
57921
58915
59909
60903
61897
62891
63885
64879
65879
66879
67879
68879
69879
70879
71879
72879
73879
74879
75879
76879
77879
78879
79879
80879
81879
82879
83879
84879
85879
86879
87879
88879
89879
90879
91879
92879
93879
94879
95879
96879
97879
98879
99869
100859
101849
102839
103829
104801
105773
106745
107717
108689
109661
110643
111625
112607
113589
114571
115571
116571
117571
118571
119571
120571
121571
122571
123571
124571
125571
126571
127571
128571
129571
130571
131571
132571
133571
134571
135571
136571
137571
138571
139571
140571
141571
142571
143571
144571
145571
146571
147571
148571
149571
150571
151571
152571
153571
154571
155571
156571
157571
158

In [58]:
data_cleaned = data.drop_duplicates()
data_indicators = data_processor.add_technical_indicators(data_cleaned,["DOT/USDT"])

Succesfully add technical indicators


In [59]:
data_indicators = data_indicators.fillna(0)
data_indicators = data_indicators.replace(np.inf,0)
#data_indicators["date"] = data_indicators.index
#data_indicators["tic"] = "DOT/USDT"

In [60]:
def split_btc(df,start,end):
    data = df[(df.index >= start) & (df.index < end)]
    #data = data.sort_values(["date", "tic"], ignore_index=True)
    return data

In [61]:
train = split_btc(data_indicators, '2021-01-01','2021-07-01')
trade = split_btc(data_indicators, '2021-07-01', '2021-08-20')

In [62]:
tech_indicator_list=[
        'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'dx_30',
        'close_30_sma', 'close_60_sma']

In [63]:
train.shape

(17321, 12)

In [67]:
stock_dimension = 1
state_space = (
    1
    + 2 * stock_dimension
    + len(tech_indicator_list) * stock_dimension
)

price_train_ary,tech_train_ary,date_train_ary = data_processor.df_to_ary(train,['DOT/USDT'])

In [65]:
e_train_gym = BitcoinEnv(price_ary=price_train_ary,tech_ary=tech_train_ary)

DatetimeIndex(['2021-01-01 02:00:00', '2021-01-01 02:15:00',
               '2021-01-01 02:30:00', '2021-01-01 02:45:00',
               '2021-01-01 03:00:00', '2021-01-01 03:15:00',
               '2021-01-01 03:30:00', '2021-01-01 03:45:00',
               '2021-01-01 04:00:00', '2021-01-01 04:15:00',
               ...
               '2021-06-30 21:30:00', '2021-06-30 21:45:00',
               '2021-06-30 22:00:00', '2021-06-30 22:15:00',
               '2021-06-30 22:30:00', '2021-06-30 22:45:00',
               '2021-06-30 23:00:00', '2021-06-30 23:15:00',
               '2021-06-30 23:30:00', '2021-06-30 23:45:00'],
              dtype='datetime64[ns]', length=17321, freq=None)